In [1]:
import os
import wget
import cv2
from utilis import CUSTOM_MODEL_NAME, PRETRAINED_MODEL_NAME, PRETRAINED_MODEL_URL, TF_RECORD_SCRIPT_NAME, LABEL_MAP_NAME, paths, files, labels

In [2]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [3]:
# https://www.tensorflow.org/install/source_windows

In [4]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 75693, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 75693 (delta 86), reused 108 (delta 56), pack-reused 75538
Receiving objects: 100% (75693/75693), 580.72 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (53725/53725), done.


### Install Tensorflow Object Detection 

In [5]:
if os.name=='posix':
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Processing /media/panjacob/ssd2/Magisterka/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Using cached avro_python3-1.10.2-py3-none-any.whl
  Using cached apache_beam-2.40.0-py3-none-any.whl
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl (352 kB)
  Using cached pycocotools-2.0.4-cp310-cp310-linux_x86_64.whl
  Using cached lvis-0.5.3-py3-none-any.whl (14 kB)
  Using cached pandas-1.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached tf_models_official-2.9.2-py2.py3-none-any.whl (2.1 MB)
  Using cached tensorflow_io-0.26.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (25.9 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached tensorflow_datasets-4.6.0-py3-none-any.whl (4.3 MB)
  Using cached kaggle-1.5.12-py3-none-a

In [6]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders',
                                   'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2022-07-29 20:00:08.486993: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/panjacob/ssd2/Magisterka/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-29 20:00:08.487013: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-29 20:00:10.349173: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/panjacob/ssd2/Magisterka/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-29 20:00:10.349192: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-29 20:00:10.349204: I tensorflow/stream_executor/cuda/cud

In [3]:
import object_detection

### Import pretrained model

In [4]:
if os.name == 'posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} & & tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} & & tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2022-07-29 20:14:38--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.203.208, 2a00:1450:401b:803::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.203.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  51.8MB/s    in 0.4s    

2022-07-29 20:14:39 (51.8 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

/bin/bash: -c: line 1: syntax error near unexpected token `&'
/bin/bash: -c: line 1: `cd Tensorflow/workspace/pre-trained-models & & tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'


# 2. Create Label Map

In [5]:
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [22]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [24]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 4. Copy Model Config to Training Folder

In [16]:
if os.name == 'posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

cp: cannot stat 'Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config': No such file or directory


# 5. Update Config For Transfer Learning

In [17]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

2022-07-29 20:25:38.474116: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/panjacob/ssd2/Magisterka/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-29 20:25:38.474135: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

NotFoundError: Tensorflow/workspace/models/my_ssd_mobnet_15000/pipeline.config; No such file or directory

In [ ]:
# config

In [73]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [74]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
# pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
# pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'train.record')]


In [75]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

# 6. Train the model

In [19]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [20]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=15000".format(TRAINING_SCRIPT,
                                                                                              paths['CHECKPOINT_PATH'],
                                                                                              files['PIPELINE_CONFIG'])

In [21]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet_15000 --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet_15000/pipeline.config --num_train_steps=15000


In [109]:
!tensorboard --logdir=./ Tensorflow / workspace / models / my_ssd_mobnet / train

^C


In [29]:
!{command}
# pip install -U tf-models-official --user

^C


# 7. Evaluate the Model

In [27]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT,
                                                                                          paths['CHECKPOINT_PATH'],
                                                                                          files['PIPELINE_CONFIG'],
                                                                                          paths['CHECKPOINT_PATH'])

In [28]:
print(command)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\my_ssd_mobnet_8000 --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet_8000\pipeline.config --checkpoint_dir=Tensorflow\workspace\models\my_ssd_mobnet_8000


In [34]:
!{command}

^C


# 8. Load Train Model From Checkpoint

In [6]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [7]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-5')).expect_partial()


@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Detect from an Image

In [13]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline

In [14]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [15]:
images = os.listdir(os.path.join(paths['IMAGE_PATH'], 'train'))
# IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'train', 'rec_0722000001_6097.png')

In [19]:
def detect_from_photo(img_path):
    img = cv2.imread(img_path)
    image_np = np.array(img)
    image_np_expanded = np.expand_dims(image_np, 0)
    input_tensor = tf.convert_to_tensor(image_np_expanded, dtype=tf.float32)
    print(image_np.shape)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=.3,
        agnostic_mode=False)

    # plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    plt.show()


for img in images:

    if img.split('.')[-1] == 'xml':
        continue
    print(img)
    detect_from_photo(os.path.join(paths['IMAGE_PATH'], 'train', img))


0_137.png
(320, 320, 3)
0_138.png
(320, 320, 3)
0_139.png
(320, 320, 3)
0_140.png
(320, 320, 3)
0_143.png
(320, 320, 3)
0_144.png
(320, 320, 3)
0_145.png
(320, 320, 3)
0_2331.png
(320, 320, 3)
0_2332.png
(320, 320, 3)
0_2551.png
(320, 320, 3)
0_3127.png
(320, 320, 3)
0_3154.png
(320, 320, 3)
0_3155.png
(320, 320, 3)
1_2281.png
(320, 320, 3)
1_2282.png
(320, 320, 3)
1_322.png
(320, 320, 3)
1_323.png
(320, 320, 3)
1_324.png
(320, 320, 3)
1_663.png
(320, 320, 3)
1_664.png
(320, 320, 3)
1_665.png
(320, 320, 3)
2_1597.png
(320, 320, 3)
2_1598.png
(320, 320, 3)
2_1884.png
(320, 320, 3)
2_2003.png
(320, 320, 3)
2_2004.png
(320, 320, 3)
3_1356.png
(320, 320, 3)
3_1357.png
(320, 320, 3)
3_1374.png
(320, 320, 3)
3_3246.png
(320, 320, 3)
3_3247.png
(320, 320, 3)
3_3280.png
(320, 320, 3)
4_351.png
(320, 320, 3)
4_352.png
(320, 320, 3)
4_368.png
(320, 320, 3)
4_462.png
(320, 320, 3)
4_463.png
(320, 320, 3)
4_464.png
(320, 320, 3)


# 10. Tracking car

In [10]:
!cd./ Tensorflow & git clone https: // github.com / nwojke / deep_sort

Cloning into 'deep_sort'...


In [26]:
from deep_sort_realtime.deepsort_tracker import DeepSort
from Tensorflow.deep_sort.application_util import preprocessing
from Tensorflow.deep_sort.deep_sort import nn_matching
from Tensorflow.deep_sort.deep_sort.detection import Detection
from Tensorflow.deep_sort.deep_sort.tracker import Tracker
from Tensorflow.deep_sort.tools import generate_detections as gdet

tracker = DeepSort(max_age=30, nn_budget=70, override_track_class=None)

vidcap = cv2.VideoCapture(os.path.join(paths['TRACK_PATH'], '1.mp4'))
count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(count):
    success, img = vidcap.read()
    if img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    image_np = np.array(img)
    image_np_expanded = np.expand_dims(image_np, 0)
    input_tensor = tf.convert_to_tensor(image_np_expanded, dtype=tf.float32)
    detections = detect_fn(input_tensor)
    print(detections)

    # bbs = object_detector.detect(frame)
    # tracks = tracker.update_tracks(bbs, frame=frame)

    cv2.imshow('hehe', img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break


AssertionError: Torch not compiled with CUDA enabled

# 10. Real Time Detections from your Webcam

In [ ]:
!pip uninstall opencv-python-headless -y

In [14]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    image_np = np.array(frame)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=.8,
        agnostic_mode=False)

    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

# 10. Freezing the Graph

In [8]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [9]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(
    FREEZE_SCRIPT, files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [10]:
print(command)

python Tensorflow\models\research\object_detection\exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet_8000\pipeline.config --trained_checkpoint_dir=Tensorflow\workspace\models\my_ssd_mobnet_8000 --output_directory=Tensorflow\workspace\models\my_ssd_mobnet_8000\export


In [35]:
# !{command}

# 11. Conversion to TFJS

In [11]:
!pip install tensorflowjs

     ---------------------------------------- 78.4/78.4 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 40.9/40.9 kB ? eta 0:00:00
  Using cached tensorflow-2.9.0-cp310-cp310-win_amd64.whl (444.1 MB)
  Using cached tensorboard-2.9.0-py3-none-any.whl (5.8 MB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.1
    Uninstalling tensorflow-2.9.1:
      Successfully uninstalled tensorflow-2.9.1

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not install packages due to an OSError: [WinError 5] Odmowa dostępu: 'C:\\Projekty SSD\\Magisterka\\venv\\Lib\\site-packages\\~ensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Check the permissions.



In [14]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(
    os.path.join(paths['OUTPUT_PATH'], 'saved_model'), paths['TFJS_PATH'])

In [15]:
print(command)

tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default Tensorflow\workspace\models\my_ssd_mobnet_8000\export\saved_model Tensorflow\workspace\models\my_ssd_mobnet_8000\tfjsexport


In [16]:
!{command}

'tensorflowjs_converter' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# Test Code: https://github.com/nicknochnack/RealTimeSignLanguageDetectionwithTFJS

# 12. Conversion to TFLite

In [39]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [40]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT,
                                                                                                         files[
                                                                                                             'PIPELINE_CONFIG'],
                                                                                                         paths[
                                                                                                             'CHECKPOINT_PATH'],
                                                                                                         paths[
                                                                                                             'TFLITE_PATH'])

In [41]:
print(command)

python Tensorflow\models\research\object_detection\export_tflite_graph_tf2.py  --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet_8000\pipeline.config --trained_checkpoint_dir=Tensorflow\workspace\models\my_ssd_mobnet_8000 --output_directory=Tensorflow\workspace\models\my_ssd_mobnet_8000\tfliteexport


In [26]:
!{command}

2021-04-03 11:55:05.530772: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:55:08.004889: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 11:55:08.005672: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2021-04-03 11:55:08.028532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2021-04-03 11:55:08.028559: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:55:08.032092: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
202

2021-04-03 11:55:14.884114: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1267]      0 
2021-04-03 11:55:14.884117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1280] 0:   N 
2021-04-03 11:55:14.884182: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 6611 MB memory) -> physical GPU (device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:2b:00.0, compute capability: 7.5)
2021-04-03 11:55:14.884197: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
W0403 11:55:15.360029 15552 save_impl.py:78] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x00000224AC2B1908>, because it is not built.
W0403 11:55:15.925411 15552 save_impl.py:78] Skipping full serialization of Keras layer <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x00000224D6D42B00>, bec

In [27]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [28]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

In [29]:
print(command)

tflite_convert --saved_model_dir=Tensorflow\workspace\models\my_ssd_mobnet\tfliteexport\saved_model --output_file=Tensorflow\workspace\models\my_ssd_mobnet\tfliteexport\saved_model\detect.tflite --input_shapes=1,300,300,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops


In [30]:
!{command}

2021-04-03 11:55:38.653963: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:55:41.159460: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 11:55:41.160164: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2021-04-03 11:55:41.183623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2021-04-03 11:55:41.183649: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:55:41.187402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
202

# 13. Zip and Export Models 

In [ ]:
!tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [17]:
from tensorflow.python.tools import freeze_graph



AttributeError: module 'tensorflow' has no attribute 'Session'

In [30]:
pb_saved_model = "/Users/vedanshu/saved_model/"
import tensorflow as tf

_graph = tf.Graph()
with _graph.as_default():
    _sess = tf.compat.v1.Session(graph=_graph)
    model = tf.compat.v1.saved_model.loader.load(_sess, ["serve"],
                                                 os.path.join('Tensorflow', 'workspace', 'models', 'my_ssd_mobnet_8000',
                                                              'export',
                                                              'saved_model'))

with tf.io.gfile.GFile("frozen.pb", "wb") as f:
    f.write(model.SerializeToString())

INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_ssd_mobnet_8000\export\saved_model\variables\variables


In [ ]:
x = tf.compat.v1.get_default_graph().get_tensor_by_name('input:0')
y = tf.compat.v1.get_default_graph().get_tensor_by_name('output:0')

export_path = './models/'
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_path)
signature = tf.saved_model.predict_signature_def(
    inputs={'input': x}, outputs={'output': y}
)

# using custom tag instead of: tags=[tf.saved_model.tag_constants.SERVING]
builder.add_meta_graph_and_variables(sess=obj.sess,
                                     tags=['serve'],
                                     signature_def_map={'predict': signature})
builder.save()